In [8]:
import re
import nltk
import hashlib
import pandas as pd

In [28]:
def clean_string(row, stem=None):
    text = row[3]
    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Keep only letters
    text = re.sub('[^a-z ]+', '', text)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']
    text_filtered = [word for word in text if not word in useless_words]

    return ' '.join(text_filtered)

In [31]:
data = pd.read_csv(
    '../dataset/switch-games-reviews.csv',
    delimiter=',',
    index_col='index'
)
data['clean_review'] = data.apply(clean_string, axis=1)
data[['id', 'clean_review']].to_csv('../dataset/switch-games-clean-reviews.csv', index=False, sep=',')